In [1]:
# install library yang dibutuhkan

! pip install googledrivedownloader

In [2]:
# Download data

from google_drive_downloader import GoogleDriveDownloader as gdd

data = [
    ["13Z3aFzWSVQXwECbP5jXOKidYf_zxfHOf", "1. Data Mahasiswa 2014 - 2019.xlsx"],
    ["1QfrrZ3MCE74yzWeTouiMD6Xuy3LLK3fT", "Data IP Sem Alumni Angkatan 2014-2019.xlsx"],
    ["1RaEodhMNIplJixsdl_ibFWv8egTbH5Gu", "NILAI TOEFL Angkatan 2014-2019.xlsx"],
    ["18E218z0-LppW3qFHAy3B9MJMqzQZ_PeQ", "penghasilan oarangtua ang 14-19.xlsx"],
]

for file_id, tittle in data:
    gdd.download_file_from_google_drive(file_id=file_id, dest_path=f"./data/{tittle}")

In [3]:
import pandas as pd

In [28]:
# Baca data

# data_ip = pd.read_excel(f"./data/{data[1][1]}")
nilai_toefl = pd.read_excel(f"./data/{data[2][1]}")
# penghasilan_ortu = pd.read_excel(f"./data/{data[3][1]}")
# data_mahasiswa = pd.read_excel(f"./data/{data[0][1]}", header=2, sheet_name=["2014", "2015", "2016", "2017", "2018", "2019"])
# data_mahasiswa = pd.concat(data_mahasiswa.values(), ignore_index=True)

In [112]:
nilai_toefl.head()

,nim,nama,Prodi,ta,sem,sem_ta,NilaiSem,n_toefl,status_akhir,Tahun_Lulus,Angkatan
0,11314001,Herlina Mariana Pardede,Teknologi Informasi,2014,1,1,1.75000,400,Lulus,2017,2014
1,11314001,Herlina Mariana Pardede,Teknologi Informasi,2014,2,2,2.45000,400,Lulus,2017,2014
2,11314001,Herlina Mariana Pardede,Teknologi Informasi,2015,3,1,2.10000,400,Lulus,2017,2014
3,11314001,Herlina Mariana Pardede,Teknologi Informasi,2015,4,2,2.66667,400,Lulus,2017,2014
4,11314001,Herlina Mariana Pardede,Teknologi Informasi,2016,5,1,2.12500,400,Lulus,2017,2014


Data preprocessing

In [118]:
# Ambil kolom "nim", "Tahun_Lulus" dan "Angkatan"
data = nilai_toefl[["nim", "Tahun_Lulus", "Angkatan"]]

# Menambahkan kolom Jumlah_Semester yang diambil dari jumlah unik dari nim nya
data["Jumlah_Semester"] = data.groupby("nim")["nim"].transform("count")

# Hapus duplikasi untuk mendapatkan data individu
data = data.drop_duplicates(subset="nim")

# Buat kolom baru "jenjang" berdasarkan digit ketiga dari "nim". D3 = 3, D4 = 2, S1 = 1.
data["jenjang"] = data["nim"].astype(str).str[2].map({"3": 3, "4": 2, "S": 1})

# Bikin class sendiri. 0 = tidak tepat waktu, 1 = tepat waktu
data["keterangan"] = "Tidak tepat waktu"

# Kondisi untuk S1
data.loc[(data["jenjang"] == 1) & (data["Jumlah_Semester"] <= 8), "keterangan"] = "Tepat waktu"

# Kondisi untuk D4
data.loc[(data["jenjang"] == 2) & (data["Jumlah_Semester"] <= 8), "keterangan"] = "Tepat waktu"

# Kondisi untuk D3
data.loc[(data["jenjang"] == 3) & (data["Jumlah_Semester"] <= 6), "keterangan"] = "Tepat waktu"

data

<ipython-input-118-41e20f0c4183>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Jumlah_Semester'] = data.groupby('nim')['nim'].transform('count')


,nim,Tahun_Lulus,Angkatan,Jumlah_Semester,jenjang,keterangan
0,11314001,2017,2014,6,3,Tepat waktu
6,11314002,2017,2014,6,3,Tepat waktu
12,11314003,2017,2014,6,3,Tepat waktu
18,11314004,2017,2014,6,3,Tepat waktu
24,11314005,2017,2014,6,3,Tepat waktu
...,...,...,...,...,...,...
16150,31S19010,2023,2019,8,1,Tepat waktu
16158,31S19011,2023,2019,8,1,Tepat waktu
16166,31S19012,2023,2019,8,1,Tepat waktu
16174,31S19013,2023,2019,10,1,Tidak tepat waktu


In [119]:
print("Jumlah tidak tepat waktu = ", len(data.loc[data["keterangan"] == "Tidak tepat waktu"]))
print("Jumlah tepat waktu = ", len(data.loc[data["keterangan"] == "Tepat waktu"]))

Jumlah tidak tepat waktu =  638
Jumlah tepat waktu =  1400


Data training LogRess

In [126]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Pisahkan variabel prediktor dan target
X = data[["Jumlah_Semester", "jenjang"]]
y = data["keterangan"]

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

# Inisialisasi dan melatih model
model = LogisticRegression()
model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)
print()
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

Akurasi: 1.0

Laporan Klasifikasi:
                   precision    recall  f1-score   support

      Tepat waktu       1.00      1.00      1.00       276
Tidak tepat waktu       1.00      1.00      1.00       132

         accuracy                           1.00       408
        macro avg       1.00      1.00      1.00       408
     weighted avg       1.00      1.00      1.00       408



In [121]:
import joblib

# Simpan model ke dalam file
joblib.dump(model, "model_logistik.pkl")

['model_logistik.pkl']

In [125]:
# Data baru yang ingin diprediksi
data_baru = {
    "Jumlah_Semester": [9],
    "jenjang": [3]
}

df_baru = pd.DataFrame(data_baru)

# Memuat kembali model dari file
model = joblib.load("model_logistik.pkl")

# Prediksi menggunakan model yang sudah dilatih
prediksi = model.predict(df_baru)

# Output hasil prediksi
print("Hasil Prediksi:", prediksi)

Hasil Prediksi: ['Tidak tepat waktu']


In [ ]:
def predict(jumlah_semester, jenjang):
    df = pd.DataFrame(
        {
            "Jumlah_Semester": [jumlah_semester],
            "jenjang": [jenjang]
        }
    )

    model = joblib.load("model_logistik.pkl")
    return model.predict(df)